In [12]:
from pymongo import MongoClient
import pandas as pd
import certifi
import json
import math
from datetime import datetime
import time
from IPython.display import clear_output
import configparser

In [13]:
# Load configuration from file
config = configparser.ConfigParser()
config.read('config.cfg')

# Access the MongoDB URI
MONGO_URI = config.get('mongo', 'MONGO_URI')
DATABASE_NAME = config.get('mongo', 'DATABASE_NAME')
COLLECTION_NAME = config.get('mongo', 'COLLECTION_NAME')
COLLECTION_NAME1 = config.get('mongo', 'COLLECTION_NAME1')

In [14]:
# Connect to the MongoDB server
client = MongoClient(MONGO_URI, tlsCAFile=certifi.where())

# Select the database and collection
db = client[DATABASE_NAME]
collection = db[COLLECTION_NAME]
collection1 = db[COLLECTION_NAME1]
# Fetch the data
data = list(collection.find())

In [15]:
# Function to read flight data from a JSON file
def read_flight_data(data):
    return json.load(data)

# Function to parse the flight data
def parse_data(flight_data):
    parsed_data = []
    for entry in flight_data:
        flight_id = entry['flight_id']
        timestamp = datetime.fromisoformat(entry['ts'])
        lat = entry['location']['lat']
        lon = entry['location']['long']
        speed = entry['velocity']['speed']
        heading = entry['velocity']['heading']
        distance_db = entry['distance_to_arrival']
        delay_length = entry['extra_length']

        parsed_data.append({
            'FlightID': flight_id,
            'Timestamp': timestamp,
            'Latitude': lat,
            'Longitude': lon,
            'Speed': speed,
            'Heading': heading,
            'Distance': distance_db,
            'Delay_length': delay_length
        })
    return parsed_data

# Function to calculate the distance between two coordinates using the Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.asin(math.sqrt(a))
    km = 6371 * c
    return km

# Function to calculate the time left to reach the destination
def time_left(distance, speed):
    if speed > 0:
        return distance / speed
    else:
        return float('inf')  # Infinite time if speed is zero

# Function to calculate the total cost
def calculate_cost(speed, distance, delay_cost_per_minute, delay_time, prev_speed, prev_fcr):
    time_left_in_hours = time_left(distance, speed)
    fuel_cost = fuel_consumption_price_to_speed(speed, prev_speed, prev_fcr)[0] * time_left_in_hours
    delay_cost = delay_cost_per_minute * delay_time
    total_cost = fuel_cost + delay_cost
    return total_cost

# A function to give fuel consumption price per hour
def fuel_consumption_price_to_speed(speed, prev_speed, prev_fcr=None):
    c = 2600 / (870**3)  # constant
    if prev_fcr is None:
        fcr_prev = c * (prev_speed**3)  # initial fuel consumption rate for the first record
    else:
        fcr_prev = prev_fcr
    fcr = fcr_prev * (prev_speed / speed)**3  # iterative calculation for subsequent records
    gallons = fcr * 2.84  # approx. 1 gallon = 2.84kg
    price = gallons * 3.5  # 1 gallon price = 3.5 EUR
    return price, fcr

# Function to process the data and calculate costs iteratively
def process_flight_data(delay_cost_per_minute):
    flight_data = data
    parsed_data = parse_data(flight_data)

    prev_fcr = None
    for i, entry in enumerate(parsed_data):
        if i == 0:
            continue

        prev_entry = parsed_data[i - 1]
        current_entry = entry

        #distance_traveled = haversine(prev_entry['Latitude'], prev_entry['Longitude'], current_entry['Latitude'], current_entry['Longitude'])
        distance_to_destination = current_entry['Distance']
        speed = current_entry['Speed']
        prev_speed = prev_entry['Speed']
        estimated_time_left = time_left(distance_to_destination, speed)

        # Calculate total cost for the current data point
        delay_time = time_left(current_entry['Delay_length'], speed)  # Assuming the delay time is the estimated time left
        price, prev_fcr = fuel_consumption_price_to_speed(speed, prev_speed, prev_fcr)
        total_cost = calculate_cost(speed, distance_to_destination, delay_cost_per_minute, delay_time, prev_speed, prev_fcr)
        delay_cost = delay_time*delay_cost_per_minute
        fuel_cost = total_cost - delay_cost
        # Clear the previous output
        clear_output(wait=True)

        # Create a document to insert into MongoDB
        document = {
            "Timestamp": current_entry['Timestamp'],
            "DistanceToDestination": distance_to_destination,
            "EstimatedTimeLeft": estimated_time_left,
            "DelayTime": delay_time,
            "DelayCost": delay_cost,
            "FuelCostPerHour": fuel_cost,
            "TotalCostPerHour": total_cost
        }

        # Insert the document into the collection
        collection1.insert_one(document)

        # Display the results
        print(f"Timestamp: {current_entry['Timestamp']}")
        #print(f"Distance Traveled: {distance_traveled:.2f} km")
        print(f"Distance to Destination: {distance_to_destination:.2f} km")
        print(f"Estimated Time Left: {estimated_time_left:.2f} hours")
        print(f"Delay Time: {delay_time:.2f} hours")
        print(f"Delay Cost: {delay_cost:.2f} EUR")
        print(f"Fuel Cost per Hour: {fuel_cost:.2f} EUR")
        print(f"Total Cost per Hour: {total_cost:.2f} EUR")
        print("-----")

        # Simulate real-time delay (e.g., 1 second)
        time.sleep(1)

# Example parameters
delay_cost_per_minute = 105  # Example delay cost per minute

# Process the flight data and calculate costs iteratively
process_flight_data(delay_cost_per_minute)




Timestamp: 2024-07-16 16:16:59.607780
Distance to Destination: 1.31 km
Estimated Time Left: 0.01 hours
Delay Time: 0.00 hours
Delay Cost: 0.00 EUR
Fuel Cost per Hour: 108.86 EUR
Total Cost per Hour: 108.86 EUR
-----
